In [ ]:
import requests
import pandas as pd
import json
import math
pd.set_option('display.max_columns', 100)

In [ ]:
r = requests.get('https://api.hh.ru/vacancies', params={'text':'Data Engineer', 'area':'1', 'per_page':100})
per_page = r.json()['per_page']
pages = r.json()['pages']

In [ ]:
vacs = []
empls = set()
for p in range(pages):    
    r = requests.get('https://api.hh.ru/vacancies', params={'page': p, 'per_page':per_page, 'text':'Data Engineer', 'area':'1'}).json()['items']
    for i in range(len(r)): 
        empls.add(r[i]['employer']['url'])
        vacs.append(r[i]['url'])

In [ ]:
det_empl = []
for empl in empls:
    det_empl.append(requests.get(empl).json())
    #if len(det_empl) == 10:
    #    break

empl_df = pd.DataFrame(det_empl)[['id', 'name', 'industries']]

In [ ]:
industries = []
for i in range(len(empl_df.id)):        
    for j in empl_df.industries[i]:        
        industries.append([empl_df.id[i], j['id'], j['name']])
         
ind_df = pd.DataFrame(industries, columns=['empl_id', 'id', 'name'])
empl_df.drop('industries', axis=1, inplace=True)

In [ ]:
det_vac = []
for vac in vacs:
    det_vac.append(requests.get(vac).json())
    if len(det_vac) % 50 == 0:
        print(f'{len(det_vac)} of {len(vacs)}')

vac_df = pd.DataFrame(det_vac)[['id', 'name', 'alternate_url', 'description', 'employer', 'employment', 'experience', 'key_skills', 'salary', 'schedule', 'specializations']]

In [ ]:
skills = []
for i in range(len(vac_df.id)):        
    for j in vac_df.key_skills[i]:        
        skills.append([vac_df.id[i], j['name']])
         
skill_df = pd.DataFrame(skills, columns=['vac_id', 'skill_name'])
vac_df.drop('key_skills', axis=1, inplace=True)

In [ ]:
vac_df = vac_df.join(pd.read_json(vac_df.salary.to_json()).T)
vac_df.drop('salary', axis=1, inplace=True)
vac_df = vac_df.join(pd.read_json(vac_df.employer.to_json()).T['id'], rsuffix='_empl')
vac_df.drop('employer', axis=1, inplace=True)
vac_df.rename(columns={'id_empl':'empl_id'}, inplace=True)

In [ ]:
def calc_salary(from_, to_, gross, curr):  
  if from_ == None:
    from_ = float('NaN')
  if to_ == None:
    to_ = float('NaN')
  if math.isnan(from_) and math.isnan(to_) or gross == None:
    res = float('NaN')
  if math.isnan(from_):
    if gross == False:
      res = to_ / 0.87
    else:
      res = to_
  elif math.isnan(to_):
    if gross == False:
      res = from_ / 0.87
    else:
      res = from_ 
  else:
    res = (from_ + to_) / 2
    if gross == False:
      res /= 0.87
  if curr == 'USD' :
    res *= 64.3
  elif curr == 'EUR':
    res *= 70.85

  return res
  
vac_df['salary'] = vac_df.apply(lambda x: calc_salary(x['from'], x['to'], x['gross'], x['currency']), axis=1)
vac_df.drop(['from', 'to', 'gross', 'currency'], axis=1, inplace=True)

In [ ]:
empl_df.to_csv('empl.csv', header=True, index=False)
ind_df.to_csv('ind.csv', header=True, index=False)
skill_df.to_csv('skills.csv', header=True, index=False)
vac_df[['id', 'name', 'salary', 'empl_id']].to_csv('vac.csv', header=True, index=False)